In [ ]:
import os
import utils
import connectome_create
# viz_method = one of ['itkwidgets', 'vtk']
viz_method = 'vtk'

# import some of our favorite packages
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import re
import warnings

%matplotlib inline
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.cluster import AgglomerativeClustering
import nglui.statebuilder as ngstbld
# from nglui.statebuilder import *

# # this is the EM specific package for querying the EM data
from caveclient import CAVEclient


# from meshparty import trimesh_io, trimesh_vtk
# from meshparty import skeletonize, skeleton_io, skeleton
# import cloudvolume

%load_ext autoreload
%autoreload 2

In [ ]:
# from pathlib import Path
# import json

# with open(Path.home() / '.cloudvolume/secrets/'/'cave-secret.json') as f:
#         tokens = json.load(f)
        
# seg_source = 'graphene://https://cave.fanc-fly.com/segmentation/table/mar2021_prod'
# cv = cloudvolume.CloudVolume(cloudpath=seg_source, use_https=True, secrets=tokens)

In [ ]:
datastack_name = 'fanc_production_mar2021'

client = CAVEclient(datastack_name)

In [ ]:
# client = CAVEclient()

# # if not os.path.isfile(os.path.expanduser("~/.cloudvolume/secrets/cave-secret.json")):
# client.auth.get_new_token(open=True)

In [ ]:
# if you have not yet setup this computer, uncomment this below line
# paste the token from the website in, and run the line

# client.auth.save_token(token="c14cd7a3e18a1a697716a399afbf5778", overwrite=True)

# then comment or delete the line as you don't need to run it on this computer  again

# Load Motor neuron table

In [ ]:
pre_to_mn_df = connectome_create.load_old_pre_to_mn_df(ext='matched_typed_with_nt')
# pre_to_mn_df = connectome_create.load_pre_to_mn_df(ext='edited')
pre_to_mn_df.shape

mpool_dict = utils.get_motor_pool_tuple_dict()
pool_keys = [
    'thorax_swing',
    'thorax_stance',
    'trochanter_extension',
    'trochanter_flexion',
    'femur_reductor',
    'tibia_extensor',
    'main_tibia_flexor',      # main_tibia_flexor for both main and A groups, or main_tibia_flexor_muscle for the muscle
    'auxiliary_tibia_flexor_A',
    'auxiliary_tibia_flexor_B',
    'auxiliary_tibia_flexor_E',
    'ltm',
    'tarsus_depressor_med_venU',
    'tarsus_depressor_noid',
    ]
pre_to_mn_df.head()

In [ ]:
%config InlineBackend.figure_formats = ['png']
%matplotlib inline

# Put pools on the motor neuron table

In [ ]:
All = slice(None)

def reorder_and_pool_motor_neurons():
    # This one adds the preferred pool to the motor neurons and appends to the pre_to_df at the top

    cclss = 'motor'
    clss_cols = pre_to_mn_df.columns.to_frame()
    clss_cols['preferred_pool'] = 'empty'

    muscle_tuple_dict = utils.get_motor_pool_tuple_dict()
    pool_keys = [
        'thorax_swing',
        'thorax_stance',
        'trochanter_extension',
        'trochanter_flexion',
        'femur_reductor',
        'tibia_extensor',
        'main_tibia_flexor',
        # 'auxiliary_tibia_flexor_A',
        'auxiliary_tibia_flexor_B',
        'auxiliary_tibia_flexor_E',
        'ltm',
        'tarsus_depressor_med_venU',
        'tarsus_depressor_noid',
        ]

    for key in pool_keys:
        mn_tup = muscle_tuple_dict[key]
        clss_cols.loc[mn_tup,'preferred_pool'] = key


    clss_cols = clss_cols.set_index('segID',drop=False)
    clss_cols['cell_class'] = 'motor'
    clss_cols.index.rename('root_id',inplace=True)
    # clss = pre_to_df.loc[(cclss,All,All,All,All,All)]
    # clss_idx  = clss.index.to_frame()
    # clss.index = clss.index.droplevel(['cell_class','preferred_pool', 'NT', 'classification_system', 'cell_type',])
    # clss_idx.index = clss_idx.index.droplevel(['cell_class','preferred_pool', 'NT', 'classification_system', 'cell_type',])

    # clss_cols has the order already figured out in previous code
    # now reduce to the premotor neurons that are still around
    # print(clss_cols.shape)
    # # clss_cols = clss_cols[clss_cols.segID.isin(clss.index.get_level_values('post_pt_root_id'))]
    # print(clss_cols.shape)

    # clss = clss.loc[clss_cols.segID]

    # # reorder the index as well
    # clss_idx = clss_idx.loc[clss_cols.segID]
    # clss_idx = clss_idx.merge(clss_cols,left_index=True,right_on='segID',how='left')
    # clss_idx

    clss_cols = clss_cols[['cell_class','preferred_pool', 'muscle', 'rank','segID']]
    clss_cols = clss_cols.rename({
            'cell_class': 'cell_class',
            'muscle': 'classification_system',
            'rank': 'cell_type',
            'segID': 'segID',
        },axis=1)
    clss_cols['NT'] = 'Glu'


    # Index has cell_class, ..., post_pt_root_id
    # clss.index = pd.MultiIndex.from_frame(clss_idx)

    # print('pre_drop:     {}'.format(pre_to_df.shape))
    # pre_to_df = pre_to_df.drop(cclss,axis=0,level='cell_class')
    # print('post_drop:    {}'.format(pre_to_df.shape))
    # pre_to_df = pd.concat([clss,pre_to_df])
    # print('post_concat:  {}'.format(pre_to_df.shape))

    # print('input order:  {}...'.format(clss_cols.segID.values[0:5]))
    # print('output order: {}...'.format(clss.index.get_level_values('post_pt_root_id').values[0:5]))

    # print('output clss order: {}'.format(pre_to_df.index.get_level_values('cell_class').unique()))

    return clss_cols

In [ ]:
clss_cols = reorder_and_pool_motor_neurons()
pre_to_mn_df.columns = pd.MultiIndex.from_frame(clss_cols)
pre_to_mn_df

# Looking for descending neurons

In [ ]:
%config InlineBackend.figure_formats = ['png']
%matplotlib inline

cmap = utils.white_dense()
def conn_heat_map(df,xticks,yticks,figsz=[16,16],savefig=False,figfilename=None,cmap=cmap):
    fig = plt.figure(1, figsize = figsz)
    ax = sns.heatmap(np.log10(df.T.to_numpy()+1), xticklabels=xticks, yticklabels=yticks, cmap=cmap)
    cbar = ax.collections[0].colorbar
    # here set the labelsize by 20
    # cbar.ax.tick_params(labelsize=20)
    cbar.set_label(label = 'log 10 # of synapses', size=24)
    plt.xlabel('Onto Output neurons', fontsize =18)
    plt.ylabel('From input neurons', fontsize =18)
    plt.yticks(fontsize = 16)
    plt.xticks(fontsize = 16)
    plt.show()
    if savefig:
        if figfilename is None:
            figfilename = 'flexor_pool_temp_' + (np.random.randint(10000,99999,size=None)).astype(str)
        fig.savefig('./figpanels/' + figfilename + '.svg',format='svg')

def lin_heat_map(df,xticks,yticks,figsz=[16,16],savefig=False,figfilename=None,cmap=cmap):
    fig = plt.figure(1, figsize = figsz)
    ax = sns.heatmap(df.T.to_numpy(), xticklabels=xticks, yticklabels=yticks, cmap=cmap)
    cbar = ax.collections[0].colorbar
    # here set the labelsize by 20
    # cbar.ax.tick_params(labelsize=20)
    cbar.set_label(label = 'linear', size=24)
    plt.xlabel('Onto Output neurons', fontsize =18)
    plt.ylabel('From input neurons', fontsize =18)
    plt.yticks(fontsize = 16)
    plt.xticks(fontsize = 16)
    plt.show()
    if savefig:
        if figfilename is None:
            figfilename = 'flexor_pool_temp_' + (np.random.randint(10000,99999,size=None)).astype(str)
        fig.savefig('./figpanels/' + figfilename + '.svg',format='svg')

In [ ]:
desc_df = pre_to_mn_df.loc['descending']
desc_df

In [ ]:
pools = 

conn_heat_map(df=desc_df, xticks=desc_df.column.get_level_values('preferred_pool'),yticks=pools,figsz=[4,16])


# Location of Femur inputs

In [ ]:
# mft = 
mft_conn_mat = pre_to_mn_df.loc[:,mpool_dict['main_tibia_flexor_muscle']]
mft_root_ids = mft_conn_mat.columns.get_level_values('segID').to_list()
mft_root_ids

## Main Tibia flexors

In [ ]:
thresh = 3
layer_name = 'flexor_input'
purples = ['#4b006e', '#7e1e9c', '#bf77f6', '#c79fef', 'eecffe']

synapse_df = client.materialize.synapse_query(post_ids = mft_root_ids,timestamp = connectome_create.get_timestamp())
v = synapse_df.pre_pt_root_id.value_counts()
synapse_df = synapse_df[synapse_df.pre_pt_root_id.isin(v.index[v.gt(thresh)])]

# variables to render in neuroglancer link

rng = np.random.default_rng()
rng_idx = rng.permutation(range(synapse_df.shape[0]))
render_synapses = synapse_df.iloc[rng_idx[0:5000],:]
# render_synapses = synapse_df
print(render_synapses.shape)
render_neurons = mft_root_ids

from nglui.statebuilder import *

img_source = client.info.image_source()
img_layer = nglui.statebuilder.ImageLayerConfig(name='fanc_v4',
                             source=img_source,
                             )
seg_source = client.info.segmentation_source()

seg_layer = nglui.statebuilder.SegmentationLayerConfig(name = 'seg',
                                    source = seg_source,
                                    fixed_ids = render_neurons)
points = nglui.statebuilder.PointMapper(point_column='post_pt_position') ######################## change this to toggle rendering of pre- or post- synaptic points
anno_layer = nglui.statebuilder.AnnotationLayerConfig(name=str(layer_name), 
                                   mapping_rules=points ) # color = purples[i]

sb = nglui.statebuilder.StateBuilder([img_layer, seg_layer, anno_layer], resolution=[4.3,4.3,45])
# sb.render_state(render_synapses, return_as='html')
nglui.statebuilder.helpers.make_url_robust(render_synapses,sb,client,shorten='always')


## Extensors Tibia 

In [ ]:
# mft = 
eti_conn_mat = pre_to_mn_df.loc[:,mpool_dict['tibia_extensor']]
eti_root_ids = eti_conn_mat.columns.get_level_values('segID').to_list()
print(eti_root_ids)

thresh = 3
layer_name = 'flexor_input'
purples = ['#4b006e', '#7e1e9c', '#bf77f6', '#c79fef', 'eecffe']

synapse_df = client.materialize.synapse_query(post_ids = eti_root_ids,timestamp = connectome_create.get_timestamp())
v = synapse_df.pre_pt_root_id.value_counts()
synapse_df = synapse_df[synapse_df.pre_pt_root_id.isin(v.index[v.gt(thresh)])]

# variables to render in neuroglancer link

rng = np.random.default_rng()
rng_idx = rng.permutation(range(synapse_df.shape[0]))
render_synapses = synapse_df.iloc[rng_idx[0:5000],:]
# render_synapses = synapse_df
print(render_synapses.shape)
render_neurons = eti_root_ids

from nglui.statebuilder import *

img_source = client.info.image_source()
img_layer = nglui.statebuilder.ImageLayerConfig(name='fanc_v4',
                             source=img_source,
                             )
seg_source = client.info.segmentation_source()

seg_layer = nglui.statebuilder.SegmentationLayerConfig(name = 'seg',
                                    source = seg_source,
                                    fixed_ids = render_neurons)
points = nglui.statebuilder.PointMapper(point_column='post_pt_position') ######################## change this to toggle rendering of pre- or post- synaptic points
anno_layer = nglui.statebuilder.AnnotationLayerConfig(name=str(layer_name), 
                                   mapping_rules=points ) # color = purples[i]

sb = nglui.statebuilder.StateBuilder([img_layer, seg_layer, anno_layer], resolution=[4.3,4.3,45])
# sb.render_state(render_synapses, return_as='html')
nglui.statebuilder.helpers.make_url_robust(render_synapses,sb,client,shorten='always')


## Aux Flexors 

In [ ]:
# mft = 
auxA_conn_mat = pre_to_mn_df.loc[:,mpool_dict['auxiliary_tibia_flexor_A']]
auxA_root_ids = auxA_conn_mat.columns.get_level_values('segID').to_list()
print(auxA_root_ids)

thresh = 3
layer_name = 'flexor_input'
purples = ['#4b006e', '#7e1e9c', '#bf77f6', '#c79fef', 'eecffe']

synapse_df = client.materialize.synapse_query(post_ids = auxA_root_ids,timestamp = connectome_create.get_timestamp())
v = synapse_df.pre_pt_root_id.value_counts()
synapse_df = synapse_df[synapse_df.pre_pt_root_id.isin(v.index[v.gt(thresh)])]

# variables to render in neuroglancer link

rng = np.random.default_rng()
rng_idx = rng.permutation(range(synapse_df.shape[0]))
render_synapses = synapse_df.iloc[rng_idx[0:5000],:]
# render_synapses = synapse_df
print(render_synapses.shape)
render_neurons = auxA_root_ids

from nglui.statebuilder import *

img_source = client.info.image_source()
img_layer = nglui.statebuilder.ImageLayerConfig(name='fanc_v4',
                             source=img_source,
                             )
seg_source = client.info.segmentation_source()

seg_layer = nglui.statebuilder.SegmentationLayerConfig(name = 'seg',
                                    source = seg_source,
                                    fixed_ids = render_neurons)
points = nglui.statebuilder.PointMapper(point_column='post_pt_position') ######################## change this to toggle rendering of pre- or post- synaptic points
anno_layer = nglui.statebuilder.AnnotationLayerConfig(name=str(layer_name), 
                                   mapping_rules=points ) # color = purples[i]

sb = nglui.statebuilder.StateBuilder([img_layer, seg_layer, anno_layer], resolution=[4.3,4.3,45])
# sb.render_state(render_synapses, return_as='html')
nglui.statebuilder.helpers.make_url_robust(render_synapses,sb,client,shorten='always')


# Proximal vs distal motor pools

In [ ]:
def build_synapses_onto_pools(render_neurons,syn_df_dict,clr_dict): # #F6921E

    img_layer = ngstbld.ImageLayerConfig(name='fanc_v4',
                                 source=client.info.image_source(),
                                 )
    seg_layer = ngstbld.SegmentationLayerConfig(name = 'seg',
                                        source =  client.info.segmentation_source(),
                                        fixed_ids = render_neurons)

    sb1 = ngstbld.StateBuilder(layers=[img_layer, seg_layer], client=client, resolution=[4.3,4.3,45])
    state_builders = [sb1]

    df_list = [None]

    for key in syn_df_dict.keys():
        print('layer {}'.format(key))
        # pool_points = ngstbld.PointMapper(point_column='pre_pt_position') ######################## change this to toggle rendering of pre- or post- synaptic points
        pool_spheres = ngstbld.SphereMapper(center_column='post_pt_position', radius_column='radius')
        pool_layer = ngstbld.AnnotationLayerConfig(name=key,
                                        mapping_rules=pool_spheres,
                                        linked_segmentation_layer=seg_layer.name,
                                        color=clr_dict[key] )
        sb_pool = ngstbld.StateBuilder(layers=[pool_layer],client=client)
        state_builders.append(sb_pool)

        df = syn_df_dict[key]
        # df.loc[:,'radius'] = int(200)
        df_list.append(df)

    sb=ngstbld.ChainedStateBuilder(statebuilders=state_builders)
    # state = sb.render_state(data_list=[ach_df,gaba_df,glu_df],return_as='dict')

    print('{} sbs, {} dfs'.format(len(state_builders),len(df_list)))

    state = sb.render_state(data_list=df_list,return_as='dict')
    state_id = client.state.upload_state_json(state)

    DEFAULT_NGL = "https://neuromancer-seung-import.appspot.com/"

    url = client.state.build_neuroglancer_url(state_id, ngl_url=DEFAULT_NGL)

    return url

In [ ]:
# Go pool by pool
All = slice(None)
timestamp = connectome_create.get_timestamp()

pool_keys = [
    # 'thorax_stance',
    'thorax_swing',
    # 'tergotrochanter',
    # 'trochanter_extensor',
    # 'extracoxal_trochanter_depressor',
    # 'trochanter_extension',
    # 'trochanter_flexion',
    # 'femur_reductor',
    # 'tibia_extensor',
    # 'main_tibia_flexor_muscle',      # main_tibia_flexor for both main and A groups, or main_tibia_flexor_muscle for the muscle
    # 'auxiliary_tibia_flexor_A',
    # 'auxiliary_tibia_flexor_B',
    # 'auxiliary_tibia_flexor_E',
    # 'ltm',
    # 'tarsus_depressor_med_venU',
    # 'tarsus_depressor_noid',
    ]
pool_clrs = {
    'thorax_stance':'#C900C7',
    'thorax_swing':'#00BFD4',
    'tergotrochanter': '#CCC000',
    'extracoxal_trochanter_depressor':'#EEC000',
    'trochanter_extensor':'#E9821D',
    'trochanter_extensor':'#FFBB8D',
    'trochanter_extension':'#203789',
    'trochanter_flexion':'#9FB7D3',
    'femur_reductor':'#EC1C24',
    'tibia_extensor':'#F6921E',
    'main_tibia_flexor_muscle':'#203789',
    'auxiliary_tibia_flexor_A':'#8CF27C',
    'auxiliary_tibia_flexor_B':'#2CEF0A',
    'auxiliary_tibia_flexor_E':'#2DB515',
    'ltm':'#FFF100',
    'tarsus_depressor_med_venU':'#CECECE',
    'tarsus_depressor_noid':'#CECECE',
}

pool_numbers = {
    'thorax_stance':'#C900C7',
    'thorax_swing':5000,
    'tergotrochanter': '#CCC000',
    'extracoxal_trochanter_depressor':'#EEC000',
    'trochanter_extensor':'#E9821D',
    'trochanter_extensor':'#E9821D',
    'trochanter_extension':'#203789',
    'trochanter_flexion':2500,
    'femur_reductor':5000,
    'tibia_extensor':5000,
    'main_tibia_flexor_muscle':2500,
    'auxiliary_tibia_flexor_A':'#8CF27C',
    'auxiliary_tibia_flexor_B':'#2CEF0A',
    'auxiliary_tibia_flexor_E':'#2DB515',
    'ltm':'#FFF100',
    'tarsus_depressor_med_venU':5000,
    'tarsus_depressor_noid':'#CECECE',
}

thresh = 3

# use a list of strings for all the html strings
ngst_dict = {}
for pk in pool_keys:    
    print(pk)
    
    mns_df = pre_to_mn_df.loc[:,mpool_dict[pk]]
    mn_root_ids = mns_df.columns.get_level_values('segID').to_list()
    print(mn_root_ids)

    syn_df_dict = {}
    clr_dict = {}
    cnt = 0;

    N_syn = 2500

    # if len(mn_root_ids)>3:
    #     mn_roots_1 = mn_root_ids[-3:]
    #     print(mn_roots_1)

    #     mn_roots_2 = mn_root_ids[:-3]
    # else:
    #     mn_roots_1 = mn_root_ids
    #     mn_roots_2 = []

    for n in reversed(mn_root_ids):
        # mn_df = mns_df.loc[:,(All,All,All,All,All,n)]
        try:
            synapse_df = client.materialize.synapse_query(post_ids = n,timestamp = connectome_create.get_timestamp())
        except ValueError:
            synapse_df = client.materialize.synapse_query(post_ids = n)
        if synapse_df.shape[0]==0:
            synapse_df = client.materialize.synapse_query(post_ids = 648518346491655185)
        v = synapse_df.pre_pt_root_id.value_counts()
        synapse_df = synapse_df[synapse_df.pre_pt_root_id.isin(v.index[v.gt(thresh)])]
        synapse_df.loc[:,'radius'] = int(100)

        # reduce the number of points in synapse_df
        rng = np.random.default_rng()
        rng_idx = rng.permutation(range(synapse_df.shape[0]))
        render_synapses = synapse_df.iloc[rng_idx[0:pool_numbers[pk]],:]
        # render_synapses = synapse_df
        print(render_synapses.shape)

        key = pk + str(n)
        syn_df_dict[key] = render_synapses
        clr_dict[key]  = pool_clrs[pk]

        cnt = cnt+1
        
        if cnt>2:
            pass
            # break

    ngst_dict[pk] = build_synapses_onto_pools(mn_root_ids,syn_df_dict,clr_dict)
    print(ngst_dict[pk])

    # syn_df_dict = {}
    # clr_dict = {}

    # if mn_roots_2:
    #     print(mn_roots_2)
    #     for n in reversed(mn_root_ids[0:-3]):
    #         # mn_df = mns_df.loc[:,(All,All,All,All,All,n)]
    #         try:
    #             synapse_df = client.materialize.synapse_query(post_ids = n,timestamp = connectome_create.get_timestamp())
    #         except ValueError:
    #             synapse_df = client.materialize.synapse_query(post_ids = n)
    #         v = synapse_df.pre_pt_root_id.value_counts()
    #         synapse_df = synapse_df[synapse_df.pre_pt_root_id.isin(v.index[v.gt(thresh)])]
    #         synapse_df.loc[:,'radius'] = int(100)

    #         # reduce the number of points in synapse_df
    #         rng = np.random.default_rng()
    #         rng_idx = rng.permutation(range(synapse_df.shape[0]))
    #         render_synapses = synapse_df.iloc[rng_idx[0:5000],:]
    #         # render_synapses = synapse_df
    #         print(render_synapses.shape)

    #         key = pk + str(n)
    #         syn_df_dict[key] = render_synapses
    #         clr_dict[key]  = pool_clrs[pk]

    #         cnt = cnt+1
            
    #         if cnt>2:
    #             pass
    #             # break
    #     pk_2 = pk + '_2'
    #     ngst_dict[pk_2] = build_synapses_onto_pools(mn_root_ids,syn_df_dict,clr_dict)
    #     print(ngst_dict[pk_2])

ngst_dict

In [ ]:
synapse_df.shape

In [ ]:
synapse_df = client.materialize.synapse_query(post_ids = n,timestamp=)